In [2]:
!pip install datasets
!pip install peft
!pip install transformers
!pip install evaluate
!pip install scikit-learn
!pip install wandb

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import evaluate
from peft import LoraConfig, get_peft_model, TaskType
import wandb
from transformers import Trainer, TrainingArguments
import evaluate

In [2]:
# Load dataset

dataset = load_dataset("emotion")

# Tokenizer and model

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Tokenize the dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=512)

# Apply tokenizer to the dataset

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [2]:
# Split the dataset into train and test

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

accuracy_metric = evaluate.load("accuracy")

# Define compute_metrics function

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Training arguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

In [ ]:
# Pretrained model evaluation

base_eval_results = trainer.evaluate()
print("Base Model:", base_eval_results)

In [2]:
from peft import LoraConfig, get_peft_model, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"]
)

peft_model = get_peft_model(model, peft_config)

In [2]:
# Update arguments

fine_tune_args = TrainingArguments(
    output_dir="./peft_model",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)

peft_trainer = Trainer(
    model=peft_model,
    args=fine_tune_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Fine-tune

peft_trainer.train()

In [2]:
# Save the model

peft_model.save_pretrained("./peft_model")

In [2]:
from peft import AutoPeftModelForSequenceClassification
from peft import PeftModel

"""
I couldn't load the Peft model using 'AutoPeftModelForSequenceClassification' because it encounters a problem
with the number of labels (6),it reverts it to the default binary (2)
"""

base_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)
fine_tuned_model = PeftModel.from_pretrained(base_model, "./peft_model")

final_trainer = Trainer(
    model=fine_tuned_model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

# Evaluate

fine_tuned_eval_results = final_trainer.evaluate()
print("Fine-Tuned Model:", fine_tuned_eval_results)